In [29]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [30]:
## Load the trained model, scaler, and encoder
model = load_model('churn_model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    encoder = pickle.load(f)
with open('onehot_encoder_geography.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

In [31]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [32]:
geo_encoded = label_encoder.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(geo_encoded, columns=label_encoder.get_feature_names_out(['Geography']))
geo_df

/home/elgendy/Projects/ANN-Project/env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [33]:
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df, geo_df], axis=1)
input_df.drop('Geography', axis=1, inplace=True)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [34]:
input_df["Gender"] = encoder.transform(input_df["Gender"])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [35]:
## Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52544045,  0.90750738,  0.10007155, -0.6962018 , -0.2629485 ,
         0.80883036,  0.64104192,  0.97067965, -0.86742511,  1.00175153,
        -0.57831252, -0.57773517]])

In [36]:
predictions = model.predict(input_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


array([[0.04838565]], dtype=float32)

In [37]:
if predictions[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
